# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\trent\Desktop\BEE4750\hw5-trentali_goatedteam`
   Installed LoggingExtras ──────── v1.1.0
   Installed GR_jll ─────────────── v0.73.8+0
   Installed JpegTurbo_jll ──────── v3.0.4+0
   Installed LERC_jll ───────────── v4.0.0+0
   Installed OffsetArrays ───────── v1.14.1
   Installed MutableArithmetics ─── v1.5.2
   Installed NetworkLayout ──────── v0.4.7
   Installed StaticArrays ───────── v1.9.8
   Installed Cairo_jll ──────────── v1.18.2+1
   Installed PlotUtils ──────────── v1.4.3
   Installed HTTP ───────────────── v1.10.10
   Installed DataFrames ─────────── v1.7.0
   Installed Libgpg_error_jll ───── v1.50.0+0
   Installed HiGHS_jll ──────────── v1.8.1+0
   Installed FFMPEG ─────────────── v0.4.2
   Installed Bzip2_jll ──────────── v1.0.8+2
   Installed SentinelArrays ─────── v1.4.7
   Installed XZ_jll ─────────────── v5.6.3+0
   Installed CodecBzip2 ─────────── v0.8.4
   Installed PlotThemes ─────────── v3.3.0
   Installed LZO_jll ────────────── v

In [5]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [51]:
using JuMP
using HiGHS

# Data
cities = [1, 2, 3]
facilities = ["LF", "MRF", "WTE"]
capacities = Dict("LF" => 200, "MRF" => 350, "WTE" => 210)  # Mg
fixed_costs = Dict("LF" => 2000, "MRF" => 1500, "WTE" => 2500)  # $/day
tipping_costs = Dict("LF" => 50, "MRF" => 7, "WTE" => 60)  # $/Mg
recycling_cost = 40  # $/Mg recycled
recycling_rate = 0.4
ash_recycled = 0.14
ash_nonrecycled = 0.16
transport_cost = 1.5  # $/Mg-Km
city_waste = Dict(1 => 100, 2 => 90, 3 => 120)  # Mg/day
distances = Dict(
    (1, "LF") => 5, (1, "MRF") => 30, (1, "WTE") => 15,
    (2, "LF") => 15, (2, "MRF") => 25, (2, "WTE") => 10,
    (3, "LF") => 13, (3, "MRF") => 45, (3, "WTE") => 20
)

# Additional inter-facility distances (limited to what was in the original problem)
inter_facility_distances = Dict(
    ("LF", "MRF") => 32, 
    ("LF", "WTE") => 18,
    ("MRF", "WTE") => 15
)

# Model setup
model = Model(HiGHS.Optimizer)

# Variables
@variable(model, W[1:3, facilities] >= 0)  # Waste from each city to each facility
@variable(model, R[facilities] >= 0)  # Residuals from each facility
@variable(model, Y[facilities], Bin)  # Binary variable for facility usage

# Objective function
@objective(model, Min, 
    sum(fixed_costs[f] * Y[f] for f in facilities) +
    sum(transport_cost * distances[c, f] * W[c, f] for c in cities, f in facilities) +
    sum(tipping_costs[f] * W[c, f] for c in cities, f in facilities) +
    recycling_cost * recycling_rate * sum(W[c, "MRF"] for c in cities) +
    # Only use inter-facility distances that are explicitly defined
    transport_cost * sum(inter_facility_distances[f1, f2] * R[f1] 
                         for (f1, f2) in keys(inter_facility_distances) if f1 != f2)
)

# City mass balance constraints
@constraint(model, city_mass_balance[c in cities], 
    sum(W[c, f] for f in facilities) == city_waste[c]
)

# Facility capacity constraints with big-M method
M = 1000
@constraint(model, facility_capacity[f in facilities], 
    sum(W[c, f] for c in cities) + (f == "MRF" ? R[f] : 0) <= capacities[f] * Y[f]
)

# MRF recycling constraints
@constraint(model, mrf_recycling, 
    R["MRF"] == 0.6 * sum(W[c, "MRF"] for c in cities)
)

# WTE ash and residual constraints
@constraint(model, wte_residuals, 
    R["WTE"] == sum(0.16 * W[c, "LF"] for c in cities) + 0.14 * R["MRF"]
)

# Facility commitment constraints
@constraint(model, facility_commitment[f in facilities], 
    M * Y[f] >= sum(W[c, f] for c in cities) + (f == "MRF" ? R[f] : (f == "WTE" ? R["WTE"] : 0))
)

# Landfill must be used
@constraint(model, landfill_required, Y["LF"] == 1)

# Solve the model
optimize!(model)

# Results
println("\nWaste Allocation:")
for c in cities
    for f in facilities
        if value(W[c, f]) > 0
            println("City ", c, " -> ", f, ": ", round(value(W[c, f]), digits=2), " Mg")
        end
    end
end

println("\nFacility Usage:")
for f in facilities
    println(f, ": ", value(Y[f]) == 1 ? "Used" : "Not Used")
end

println("\nResiduals:")
for f in facilities
    if value(R[f]) > 0
        println(f, " Residual: ", round(value(R[f]), digits=2), " Mg")
    end
end

println("\nTotal Cost: ", round(objective_value(model), digits=2))

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 1e+03]
  Cost   [2e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Presolving model
10 rows, 13 cols, 40 nonzeros  0s
8 rows, 11 cols, 34 nonzeros  0s
7 rows, 10 cols, 23 nonzeros  0s

Solving MIP model with:
   7 rows
   10 cols (1 binary, 0 integer, 0 implied int., 9 continuous)
   23 nonzeros
MIP-Timing:      0.0002 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp C

## References

List any external references consulted, including classmates.